<a href="https://colab.research.google.com/github/patrycjaskowronska/UMWF/blob/main/case3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install backtesting

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173916 sha256=8ab9173131b532d66b83ac2b4e853f96bb292fabe8b991c10570f424f2f32a7a
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting


In [2]:
! pip install keras_tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.1 MB/s eta 0:00:00


In [3]:
import yfinance as yf
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
import keras_tuner
from keras_tuner.tuners import RandomSearch
from backtesting import Backtest, Strategy

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


In [4]:
data = yf.download('AMZN',
                   start='2024-01-01',
                   end='2024-05-06')

data.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-01-02,151.539993,152.380005,148.389999,149.929993,149.929993,47339400
2024-01-03,149.199997,151.050003,148.330002,148.470001,148.470001,49425500
2024-01-04,145.589996,147.380005,144.050003,144.570007,144.570007,56039800
2024-01-05,144.690002,146.589996,144.529999,145.240005,145.240005,45124800
2024-01-08,146.740005,149.399994,146.149994,149.100006,149.100006,46757100


In [5]:
# podział na zbiór treningowy i testowy

data['Target'] = data['Adj Close'].shift(-1)
data = data.dropna()

X = data[['Open', 'High', 'Low', 'Close', 'Volume']]
y = data['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [6]:
# standaryzacja

scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

In [7]:
# budowa modelu + optymalizacja hiperparametrów

def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Choice('units', [8, 16, 32]), activation='relu', input_dim=X_train_scaled.shape[1]))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))
    model.compile(optimizer='Adam', loss='mean_squared_error')
    return model

tuner = keras_tuner.RandomSearch(build_model, objective='val_loss', max_trials=5, executions_per_trial=1, directory='tuner_results')

tuner.search(X_train_scaled, y_train_scaled, epochs=5, validation_data=(X_test_scaled, y_test_scaled))

best_model = tuner.get_best_models(num_models=1)[0]

Trial 3 Complete [00h 00m 02s]
val_loss: 0.9985784888267517

Best val_loss So Far: 0.8389182090759277
Total elapsed time: 00h 00m 05s


In [8]:
data['Prediction'] = scaler_y.inverse_transform(best_model.predict(scaler_X.transform(X)))

data['Signal'] = 0
data['Signal'] = np.where(data['Prediction'].shift(-1) > data['Prediction'], 1, 0)
data['Position'] = data['Signal'].shift()

data_bt = data[['Open', 'High', 'Low', 'Close', 'Volume', 'Signal', 'Position']].copy()
data_bt.dropna(inplace=True)

3/3 [==============================] - 0s 4ms/step


<ipython-input-8-2c694abda013>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Prediction'] = scaler_y.inverse_transform(best_model.predict(scaler_X.transform(X)))
<ipython-input-8-2c694abda013>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Signal'] = 0
<ipython-input-8-2c694abda013>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

In [9]:
# backtesting

class AmazonStrategy(Strategy):
    def init(self):
        self.signal = self.data.Signal

    def next(self):
        price = self.data.Close
        long_sl = price * 0.99
        long_tp = price * 1.05
        short_sl = price * 1.01
        short_tp = price * 0.95

        if self.signal == 1:
            if self.position.is_short:
                self.position.close()
                self.buy(size=0.6, sl=long_sl, tp=long_tp)
            elif not (self.position.is_short or self.position.is_long):
                self.buy(size=0.6, sl=long_sl, tp=long_tp)

        elif self.signal == 0:
            if self.position.is_long:
                self.position.close()
                self.sell(size=0.6, sl=short_sl, tp=short_tp)
            elif not (self.position.is_short or self.position.is_long):
                self.sell(size=0.6, sl=short_sl, tp=short_tp)



bt = Backtest(data_bt, AmazonStrategy, cash=10000, exclusive_orders=True)
output = bt.run()
bt.plot()
print(output)

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


Start                     2024-01-03 00:00:00
End                       2024-05-02 00:00:00
Duration                    120 days 00:00:00
Exposure Time [%]                   97.619048
Equity Final [$]                  8608.826469
Equity Peak [$]                       10000.0
Return [%]                         -13.911735
Buy & Hold Return [%]               24.415707
Return (Ann.) [%]                  -36.198357
Volatility (Ann.) [%]                8.428011
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -16.989703
Avg. Drawdown [%]                  -16.989703
Max. Drawdown Duration      119 days 00:00:00
Avg. Drawdown Duration      119 days 00:00:00
# Trades                                   34
Win Rate [%]                         5.882353
Best Trade [%]                       5.068822
Worst Trade [%]                      -5.28314
Avg. Trade [%]                    